In [26]:
!apt-get install tesseract-ocr
!pip install nltk scikit-learn PyPDF2 pdf2image pytesseract pillow


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.


In [2]:
import nltk
import re
import PyPDF2
import pandas as pd
import numpy as np
import pytesseract

from pdf2image import convert_from_path
from PIL import Image
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
from google.colab import files
uploaded = files.upload()


Saving imageprocessor.pdf to imageprocessor.pdf


In [15]:
pdf_path = list(uploaded.keys())[0]

images = convert_from_path(pdf_path)

text = ""
for img in images:
    text += pytesseract.image_to_string(img)

text[:1000]


'ewe! downtown @\n\n‘SUUrvuuvgeecuuavaceuanyeecvoogeecueveeneacouzzuaegc even ceeeee cece\n\nUNS Ta State’s IDPS unit honoured Gujarat Skill Devpt\n\nState emerging asa Secy visits WSC\n\n \n\nST NewS NETWORK\nL Rajendra Pradhan, Bhubaneswar, Nov 4: Vinod R\nS/o- Srikanta Charan Rao, Secretary of Labour, Skill\n\nrt Development and Employment\nPradhan, resident of e . Serj, of Gujarat government, visited\nward No.- 6, Benuda, “S* the World Skill Center (WSC)\nP.O. Katisahi, P.S. e NEWS NETWORK ment ofdedication andcommit- here, Monday, as part of his\n\narda ment of the team in safeguard: two-day visit to the state,\nKamarda, Dist Bhubaneswar, Nov 4: The ing the public health in Odish: Rao explored various state-\n\n  \n\n   \n\n \n\n \n\n   \n\n   \n\n \n\nBaleshwar, Pin- 756035, The Governor Odisha unit of Integrated Dis- Notably, the Sate Surveil- of-the-art laboratories and in-\nisha declare by this aoe ease Surveillance Programme lance Unit functions round the —_teracted with stud

In [17]:
stop_words = set(stopwords.words("english"))

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^a-zA-Z ]', '', text)
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [w for w in tokens if w not in stop_words]
    return " ".join(tokens)

cleaned_text = clean_text(text)
sentences = sent_tokenize(text)


In [18]:
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(sentences)

sentence_scores = np.array(tfidf_matrix.sum(axis=1)).ravel()
top_sentences = sentence_scores.argsort()[-5:]

summary = " ".join([sentences[i] for i in sorted(top_sentences)])
summary


'e NEWS NETWORK ment ofdedication andcommit- here, Monday, as part of his\n\narda ment of the team in safeguard: two-day visit to the state,\nKamarda, Dist Bhubaneswar, Nov 4: The ing the public health in Odish: Rao explored various state-\n\n  \n\n   \n\n \n\n \n\n   \n\n   \n\n \n\nBaleshwar, Pin- 756035, The Governor Odisha unit of Integrated Dis- Notably, the Sate Surveil- of-the-art laboratories and in-\nisha declare by this aoe ease Surveillance Programme lance Unit functions round the —_teracted with students on the\n\nOdisha declare by thi\n\naffidavit No.- 169/2024 highlighted that (IDSP) was honoured for its clock during natural disasters advanced skill courses offeres! “The WSC is\n\na vices (IDSP) and the S officials, making a significant contribution\nalso develop essential veitiance oicer (S Also, it regularly monitors to the development of Odisha by\nattributes such Kumar Paik: the real time disease data en- _providinga platform for talented\nis aiworK and honouron behal

In [21]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(sentences)

kmeans = KMeans(n_clusters=3, random_state=42, n_init='auto')
kmeans.fit(X)

terms = vectorizer.get_feature_names_out()
centroids = kmeans.cluster_centers_

for i in range(3):
    top_terms_indices = centroids[i].argsort()[-10:]
    print(f"Topic {i+1}: ", [terms[j] for j in top_terms_indices])

Topic 1:  ['progressive', 'women', 'tunities', 'birabar', 'bath', 'late', 'continues', 'contrit', 'thrive', 'trade']
Topic 2:  ['hours', 'activity', 'increase', 'insiya', 'daily', 'taking', 'changed', 'son', 'minutes', 'risk']
Topic 3:  ['fish', 'state', 'river', 'odisha', 'health', 'purposes', 'world', 'meeting', 'hindi', 'said']


In [25]:
print("=========== NEWS SUMMARY ===========\n")
print(summary)


=========== NEWS SUMMARY ===========

e NEWS NETWORK ment ofdedication andcommit- here, Monday, as part of his

arda ment of the team in safeguard: two-day visit to the state,
Kamarda, Dist Bhubaneswar, Nov 4: The ing the public health in Odish: Rao explored various state-

  

   

 

 

   

   

 

Baleshwar, Pin- 756035, The Governor Odisha unit of Integrated Dis- Notably, the Sate Surveil- of-the-art laboratories and in-
isha declare by this aoe ease Surveillance Programme lance Unit functions round the —_teracted with students on the

Odisha declare by thi

affidavit No.- 169/2024 highlighted that (IDSP) was honoured for its clock during natural disasters advanced skill courses offeres! “The WSC is

a vices (IDSP) and the S officials, making a significant contribution
also develop essential veitiance oicer (S Also, it regularly monitors to the development of Odisha by
attributes such Kumar Paik: the real time disease data en- _providinga platform for talented
is aiworK and honour